In [1]:
#!/usr/bin/env python
import snowflake.connector
import time
import hashlib as hashlib
import pandas as pd
import numpy as np
import calendar
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
from hashlib import md5

###
# https://docs.snowflake.com/en/user-guide/python-connector-example.html
###
from datetime import date, datetime, timedelta
from dateutil.parser import parse as dateParse

d:\SnowFlake\Labs


In [2]:
#---------------------------------------
# CONNECT TO SNOWFLAKE
#---------------------------------------

%cd ..
import credentials
# Create main connection context
cxn = snowflake.connector.connect(
    user=credentials.user, # username
    password=credentials.password, # password
    account=credentials.account, # ********.central-india.azure
    
    #extras 
    warehouse=credentials.warehouse,
    database=credentials.database, #DEMO_DB
    schema=credentials.schema
#     , protocol='https'
#     , port=port
#     ,
#     session_parameters={
#     'QUERY_TAG': 'EndOfMonthFinancials',
#     }
    )
# -- End Connection Creation

# CREATE USER snowflakemeetup20210524 PASSWORD = '*************' 
# COMMENT = 'Delete After the Event' LOGIN_NAME = 'snowflakemeetup20210524' 
# DISPLAY_NAME = 'AICG - We Love Data' DEFAULT_ROLE = "DATALAKEHOUSE_ROLE" DEFAULT_WAREHOUSE = 'FIVETRAN_WAREHOUSE' 
# MUST_CHANGE_PASSWORD = FALSE;
# GRANT ROLE "DATALAKEHOUSE_ROLE" TO USER snowflakemeetup20210524;


In [3]:
#---------------------------------------
# GET CURRENT VERSION OF SNOWFLAKE
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()
try:
    # Gets the version
    cxn_cursor.execute("SELECT current_version(), current_role(), current_database();")
    
    # Gets the first column and record returned from the cursor and returns
    one_row = cxn_cursor.fetchone()
    print(one_row)
    
    #alternatively if returning two columns then assign in dict
    #col1, col2 = con.cursor().execute("SELECT col1, col2 FROM testtable").fetchone()
    #print('{0}, {1}'.format(col1, col2))
    
    
finally:
    cxn_cursor.close()
    

('6.32.3', 'ACCOUNTADMIN', 'SNOWFLAKE_SAMPLE_DATA')


In [4]:
#---------------------------------------
# Basic Retrieval from the Customer Dataset from SNOWFLAKE_SAMPLE_DATA
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()
try:
    # Gets the data via SQL Select Query
    cxn_cursor.execute("SELECT C_CUSTKEY, C_ACCTBAL, C_NAME, C_NATIONKEY, C_MKTSEGMENT FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER limit 100;")
    
    # Loops/iterates through the cursor
    for rec in cxn_cursor:
        print('%s, %s, %s, %s, %s' % (rec[0], rec[1], rec[2], rec[3], rec[4]))
        
    # Snowflake execute alternatively returns an iterator and can be called as the for loop
    #for (col1, col2) in con.cursor().execute("SELECT col1, col2 FROM testtable"):
    #    print('{0}, {1}'.format(col1, col2))
    
finally:
    cxn_cursor.close()

60001, 9957.56, Customer#000060001, 14, HOUSEHOLD
60002, 742.46, Customer#000060002, 15, BUILDING
60003, 2526.92, Customer#000060003, 16, BUILDING
60004, 7975.22, Customer#000060004, 10, AUTOMOBILE
60005, 2504.74, Customer#000060005, 12, MACHINERY
60006, 9051.40, Customer#000060006, 22, MACHINERY
60007, 6017.17, Customer#000060007, 12, FURNITURE
60008, 5621.44, Customer#000060008, 2, AUTOMOBILE
60009, 9548.01, Customer#000060009, 9, FURNITURE
60010, 3497.91, Customer#000060010, 21, HOUSEHOLD
60011, 3422.48, Customer#000060011, 8, BUILDING
60012, 671.60, Customer#000060012, 23, FURNITURE
60013, -485.69, Customer#000060013, 0, MACHINERY
60014, 7932.15, Customer#000060014, 16, FURNITURE
60015, 4622.39, Customer#000060015, 9, FURNITURE
60016, 4480.97, Customer#000060016, 2, HOUSEHOLD
60017, 3653.17, Customer#000060017, 1, AUTOMOBILE
60018, 5759.83, Customer#000060018, 0, BUILDING
60019, 3444.77, Customer#000060019, 1, MACHINERY
60020, 1187.30, Customer#000060020, 15, FURNITURE
60021, 9919.

In [5]:
#---------------------------------------
# PANDAS DataFrame Retrieval from the Customer Dataset
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()

try:
    # Gets the SQL via SELECT Query
    cxn_cursor.execute("SELECT C_CUSTKEY, C_ACCTBAL, C_NAME, C_NATIONKEY, C_MKTSEGMENT FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER limit 1000;")

    # Loops/iterates through the cursor
#     rows = 0
#     while True:
#         data_output = cxn_cursor.fetchmany(5000)
#         if not data_output:
#             break
#         pdf = pd.DataFrame(data_output, columns=cxn_cursor.description)
#         rows += pdf.shape[0]
    #print(rows)
    # pd.set_option("max_columns", 2)
    df = cxn_cursor.fetch_pandas_all()
    
#     print(df)
    
finally:
    cxn_cursor.close()
    
df.head(5)

,C_CUSTKEY,C_ACCTBAL,C_NAME,C_NATIONKEY,C_MKTSEGMENT
0,60001,9957.56,Customer#000060001,14,HOUSEHOLD
1,60002,742.46,Customer#000060002,15,BUILDING
2,60003,2526.92,Customer#000060003,16,BUILDING
3,60004,7975.22,Customer#000060004,10,AUTOMOBILE
4,60005,2504.74,Customer#000060005,12,MACHINERY


In [6]:
#---------------------------------------
# CREATE A SCHEMA
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()
try:
    # Switch Context
    cxn_cursor.execute("USE DATABASE MYDB")
    cxn_cursor.execute("CREATE SCHEMA IF NOT EXISTS PYTHONDATA")
    cxn_cursor.execute("USE SCHEMA PYTHONDATA")
    

    # Gets the version
    cxn_cursor.execute("SELECT current_version()")
    
    # Gets the first record returned from the cursor and returns
    one_row = cxn_cursor.fetchone()
    print(one_row[0])
    
finally:
    cxn_cursor.close()

6.32.3


In [7]:
#---------------------------------------
# CREATE A TABLE AND INSERT SOME RECORDS
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()
try:
    # Creates the Table
    cxn_cursor.execute("CREATE OR REPLACE TABLE MEETUP_GUESTS (guest_name string, guest_phone varchar(12), guest_interests varchar(1000))")
    
    # Insert a record into the table to confirm table exists
    cxn_cursor.execute("""
        INSERT INTO MEETUP_GUESTS (guest_name, guest_phone, guest_interests) 
        VALUES ('Elon Musk', '888-SKY-HIGH', 'Rockets, Cars, Space, Cryto, DataLakeHouse.io')
    """ 
    )
    
    # Just a validation to make sure the script didn't abend/crash before here
    cxn_cursor.execute("SELECT current_version()")
    
    # Get the records from the table
    cxn_cursor.execute("SELECT * FROM MEETUP_GUESTS")
    
    print("Here is the Snowflake Query ID that handled the above cursor execution")
    print(cxn_cursor.sfqid)
    
    print("")

    # Print
    df = cxn_cursor.fetch_pandas_all()
    
    print(df.head(100))
    
finally:
    cxn_cursor.close()

Here is the Snowflake Query ID that handled the above cursor execution
01a79ef1-0000-dcbe-0000-000085a5493d

  GUEST_NAME   GUEST_PHONE                                GUEST_INTERESTS
0  Elon Musk  888-SKY-HIGH  Rockets, Cars, Space, Cryto, DataLakeHouse.io


In [8]:
#---------------------------------------
# INSERT RECORDS WITH A EXECUTEMANY()
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()

nData = df

try:
    # New DataFrame with More Guests Names

    # dictionary with list object in values
    #df = pd.DataFrame([('Albert', 'E', 'Theories, Collectibles, Science'), ('Nikoli', 'Tesla', 'Eletricity, Winning, Soft Noises, Ideas')], columns=['guest_name', 'guest_phone', 'guest_interests'])
    
#     rows_to_insert = [('Albert E.', '888-ALBERTE', 'Theories, Collectibles, Science'), ('Nikoli Tesla', '555-RDE-LTNG', 'Eletricity, Winning, Soft Noises, Ideas')]
#     cxn_cursor.execute(
#         "insert into MEETUP_GUESTS (guest_name, guest_phone, guest_interests) values (?, ?, ?)",
#         rows_to_insert)

    #INSERT PLAINLY One LINE AT A TIME
    cxn_cursor.execute("INSERT INTO MEETUP_GUESTS VALUES('Albert E.', '888-ALBERTE', 'Theories, Collectibles, Science')")
    cxn_cursor.execute("INSERT INTO MEETUP_GUESTS VALUES('Nikoli Tesla', '555-RDE-LTNG', 'Eletricity, Winning, Soft Noises, Ideas')")

    
    print("")

    # Get the records from the table
    nData = cxn_cursor.execute("SELECT * FROM MEETUP_GUESTS").fetch_pandas_all()
    
    print("Here is the Snowflake Query ID that handled the above cursor execution")
    print(cxn_cursor.sfqid)
    
    print("")
    
    
except snowflake.connector.errors.ProgrammingError as e:
    # default error message
    print(e)
    # customer error message
    print('Error {0} ({1}): {2} ({3})'.format(e.errno, e.sqlstate, e.msg, e.sfqid))
    
    # ROLLBACK any part of this transaction
    cxn_cursor.rollback()
    raise e
finally:
    cxn_cursor.close()
    
nData.head(5)



Here is the Snowflake Query ID that handled the above cursor execution
01a79ef9-0000-dcb8-0000-000085a53929



,GUEST_NAME,GUEST_PHONE,GUEST_INTERESTS
0,Elon Musk,888-SKY-HIGH,"Rockets, Cars, Space, Cryto, DataLakeHouse.io"
1,Albert E.,888-ALBERTE,"Theories, Collectibles, Science"
2,Nikoli Tesla,555-RDE-LTNG,"Eletricity, Winning, Soft Noises, Ideas"


In [9]:
#---------------------------------------
# LOAD DATA FROM FILES ON SERVER OR LAPTOP
#---------------------------------------

# https://en.wikipedia.org/wiki/Cursor_(databases)
cxn_cursor = cxn.cursor()

nData = df

try:

    # Use the PUT Operation into an EXISTING STAGE by the name of, 
    #cxn_cursor.execute("PUT file:\/Users\/christian\/Coding\/tmp\/meetup_guest2.csv @default_loads")
#     cxn_cursor.execute("COPY INTO default_loads")
    
#     print("")

#     # Get the records from the table
#     nData = cxn_cursor.execute("SELECT * FROM MEETUP_GUESTS").fetch_pandas_all()
    
#     print("Here is the Snowflake Query ID that handled the above cursor execution")
    print(cxn_cursor.sfqid)
    
    print("")
    
    
except snowflake.connector.errors.ProgrammingError as e:
    # default error message
    print(e)
    # customer error message
    print('Error {0} ({1}): {2} ({3})'.format(e.errno, e.sqlstate, e.msg, e.sfqid))
    
    # ROLLBACK any part of this transaction
    #cxn_cursor.rollback()
    raise e
finally:
    cxn_cursor.close()
    
nData.head(5)

None



,GUEST_NAME,GUEST_PHONE,GUEST_INTERESTS
0,Elon Musk,888-SKY-HIGH,"Rockets, Cars, Space, Cryto, DataLakeHouse.io"


In [10]:
#---------------------------------------
# CLOSE CONNECTION TO SNOWFLAKE
# -- Will require reconnecting/authorizing in the top most cell for connecting
#---------------------------------------
if cxn:
    cxn.close()